In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
data = pd.read_csv("../dataset/emotions.csv")

# Data preprocessing
X = data['text']
y = data['label']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Padding sequences
max_length = 100  # Define your maximum sequence length
X = pad_sequences(X, maxlen=max_length)

# Convert labels to one-hot encoding
num_classes = len(data['label'].unique())
y = pd.get_dummies(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Predict labels
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(np.array(y_test), axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)


Epoch 1/10
8337/8337 ━━━━━━━━━━━━━━━━━━━━ 710s 85ms/step - accuracy: 0.8243 - loss: 0.4460 - val_accuracy: 0.9369 - val_loss: 0.1027
Epoch 2/10
8337/8337 ━━━━━━━━━━━━━━━━━━━━ 735s 88ms/step - accuracy: 0.9396 - loss: 0.1002 - val_accuracy: 0.9390 - val_loss: 0.0976
Epoch 3/10
8337/8337 ━━━━━━━━━━━━━━━━━━━━ 743s 89ms/step - accuracy: 0.9429 - loss: 0.0900 - val_accuracy: 0.9362 - val_loss: 0.0945
Epoch 4/10
8337/8337 ━━━━━━━━━━━━━━━━━━━━ 778s 93ms/step - accuracy: 0.9447 - loss: 0.0850 - val_accuracy: 0.9355 - val_loss: 0.0991
Epoch 5/10
8337/8337 ━━━━━━━━━━━━━━━━━━━━ 762s 91ms/step - accuracy: 0.9470 - loss: 0.0822 - val_accuracy: 0.9328 - val_loss: 0.1025
Epoch 6/10
8337/8337 ━━━━━━━━━━━━━━━━━━━━ 743s 89ms/step - accuracy: 0.9508 - loss: 0.0785 - val_accuracy: 0.9282 - val_loss: 0.1060
2606/2606 ━━━━━━━━━━━━━━━━━━━━ 56s 22ms/step - accuracy: 0.9386 - loss: 0.0941
Test Loss: 0.09458000957965851
Test Accuracy: 0.9378253817558289
2606/2606 ━━━━━━━━━━━━━━━━━━━━ 55s 21ms/step
Precision: 0.

In [9]:
model.save('emotions_rnn_new.keras')

In [6]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
data = pd.read_csv("../dataset/emotions.csv")

# Data preprocessing
X = data['text']
y = data['label']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Padding sequences
max_length = 100  # Define your maximum sequence length
X = pad_sequences(X, maxlen=max_length)

# Convert labels to one-hot encoding
num_classes = len(data['label'].unique())
y = pd.get_dummies(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


saved_model = load_model('emotions_rnn_new.keras')

loss, accuracy = saved_model.evaluate(X_test, y_test)

# 4. Make predictions using the model
y_pred = saved_model.predict_classes(X_test)

# 5. Calculate additional metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

/home/mizhap/.local/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 20 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 1 objects could not be loaded. Example error message for object <LSTMCell name=lstm_cell, built=True>:

Layer 'lstm_cell' expected 3 variables, but received 0 variables during loading. Expected: ['kernel', 'recurrent_kernel', 'bias']

List of objects that could not be loaded:
[<LSTMCell name=lstm_cell, built=True>]

In [8]:
from keras.models import load_model

saved_model = load_model('emotions_rnn_new.keras')

loss, accuracy = saved_model.evaluate(X_test, y_test)

# 4. Make predictions using the model
y_pred = saved_model.predict_classes(X_test)

# 5. Calculate additional metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

ValueError: A total of 1 objects could not be loaded. Example error message for object <LSTMCell name=lstm_cell, built=True>:

Layer 'lstm_cell' expected 3 variables, but received 0 variables during loading. Expected: ['kernel', 'recurrent_kernel', 'bias']

List of objects that could not be loaded:
[<LSTMCell name=lstm_cell, built=True>]